# Import and Constract Dataset

In [8]:
import numpy as np
import scipy.optimize
from scipy.optimize import minimize

root_dir: str = "Lab2//data//"
def construct_dataset():
    m = 150  # data rows
    d = 75  # feature dimensions
    X: np.ndarray = np.random.rand(m, d)
    theta: np.ndarray = np.zeros(shape=(d, 1))

    theta[:10] = np.array([10 if np.random.randint(0, 2) == 0 else -10 for _ in range(10)]).reshape((10, 1))

    epsilon: np.ndarray = np.random.normal(loc=0, scale=0.1, size=(m, 1))
    y: np.ndarray = np.dot(X, theta) + epsilon
    
    np.savetxt(root_dir + "X_train.txt", X[:80])
    np.savetxt(root_dir + "X_validation.txt", X[80:100])
    np.savetxt(root_dir + "X_test.txt", X[100:])
    np.savetxt(root_dir + "X.txt", X[:])
    np.savetxt(root_dir + "Y_train.txt", y[:80])
    np.savetxt(root_dir + "Y_validation.txt", y[80:100])
    np.savetxt(root_dir + "Y_test.txt", y[100:])
    np.savetxt(root_dir + "Y.txt", y[:])


construct_dataset()


# Ridge Regression

In [10]:
def ridge_regression_test():
    def ridge(Lambda):
        def ridge_obj(theta):
            return ((np.linalg.norm(np.dot(X_train, theta) - y_train)) ** 2) \
                   / (2 * N) + Lambda * (np.linalg.norm(theta)) ** 2

        return ridge_obj

    def compute_loss(Lambda, theta):
        return ((np.linalg.norm(np.dot(X_valid, theta) - y_valid)) ** 2) / (2 * N)

    X_train = np.loadtxt(root_dir + "X_train.txt")
    X_valid = np.loadtxt(root_dir + "X_validation.txt")
    y_train = np.loadtxt(root_dir + "Y_train.txt")
    y_valid = np.loadtxt(root_dir + "Y_validation.txt")

    (N, D) = X_train.shape
    w = np.random.rand(D)

    min_lambda = 0
    min_loss = 1e100
    min_opt_result: scipy.optimize.OptimizeResult
    for i in range(-5, 6):
        Lambda = 10 ** i
        w_opt = minimize(ridge(Lambda), w)
        loss = compute_loss(Lambda, w_opt.x)
        if loss < min_loss:
            min_loss = loss
            min_opt_result = w_opt
            min_lambda = Lambda
        print("lambda: ", Lambda, "loss: ", loss)
    print("lambda: ", min_lambda, "min loss", min_loss)
    print("theta result is: \n", min_opt_result.x)

    true_zero_count = 0
    thresh_hold = 1e-3
    small_count = 0
    for ele in min_opt_result.x:
        if ele == 0:
            true_zero_count = true_zero_count + 1
        elif ele <= thresh_hold:
            small_count = small_count + 1
    print("True zero count is {0}, values smaller than {1} is {2}"
          .format(true_zero_count, thresh_hold, small_count))


ridge_regression_test()

lambda:  1e-05 loss:  0.008713433584521039
lambda:  0.0001 loss:  0.21215329964425256
lambda:  0.001 loss:  0.9052485084699834
lambda:  0.01 loss:  2.008252294219313
lambda:  0.1 loss:  5.319148897642792
lambda:  1 loss:  7.52612187475231
lambda:  10 loss:  8.82599214386983
lambda:  100 loss:  13.176956788171845
lambda:  1000 loss:  14.407460974371187
lambda:  10000 loss:  14.549752458880935
lambda:  100000 loss:  14.56419938076532
lambda:  1e-05 min loss 0.008713433584521039
theta result is: 
 [-9.89207587e+00  9.79697225e+00 -9.77003135e+00  9.99556613e+00
  9.90628412e+00  9.92926202e+00  9.80449422e+00 -1.00572693e+01
 -1.01368215e+01  1.00989530e+01  1.84867181e-01  1.18619597e-01
 -7.97759808e-02 -3.35754263e-01  1.70214002e-01  4.13195617e-02
  5.47307209e-02  1.62624666e-01 -2.47763306e-03  5.87920743e-02
 -1.81911270e-03 -9.26412251e-02 -5.95487400e-02  1.88576702e-02
 -9.68252153e-02 -1.67602253e-01 -9.14761259e-02 -8.46451030e-02
 -1.17419686e-01  1.16961698e-01  1.78351201e